# Importing Modules and Loading Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dataCleaner import preProcess
from helper_fns import lexicon_calculate
import nltk

%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\youss\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
tweets_df = pd.read_csv('Datasets/tweets_sns2.csv')
tweets_df.head()

,user_name,user_location,text,likes,date
0,klbe1997,الوادي الجديد,شعري لونه orange وانتوا؟!,0,2022-08-28
1,SaMar2555_,الوادي الجديد,متعتك عندي بتحويل فودافون كاش سكس فون و كاميرا...,2,2022-08-28
2,dyna86889772,الوادي الجديد,شعري لونه orange وانتوا؟!,0,2022-08-28
3,hcintma,الوادي الجديد,شعري لونه orange وانتوا؟!,0,2022-08-28
4,alikreek564,الوادي الجديد,شعري لونه orange وانتوا؟!,1,2022-08-28


In [3]:
tweets_df.shape

(5384, 5)

# Cleaning and Preprocessing the Data

In [29]:
keep_regex = "[Vv]odafone|VODAFONE|فودافون|[Ee]tisalat|ETISALAT|اتصالات|[Oo]range|ORANGE|اورانج|موبينيل|إتصالات|أورانج"
remove_regex = "لون"

In [30]:
drop_indeces = tweets_df[(tweets_df['text'].str.contains(keep_regex)==False) | (tweets_df['text'].str.contains(remove_regex)==True)].index

In [31]:
tweets_df.drop(drop_indeces, inplace=True)
tweets_df.shape

(4941, 5)

In [7]:
tweets_df.iloc[1400, 2]

'إعلان vodafone Egypt : راجعين للكيان \nأعلان etisalat misr : بنحب الأهلي \nأعلان we Egypt : معاك يا أهلي \n\nالزمالك : طب اعلان واحد حتي 💔\n#معاك_يا_اهلي'

In [8]:
tweets_df.iloc[1300, 2]

'شبكة اتصالات واقعه عن نص طنطا 😂😂'

In [32]:
tweets_df = tweets_df.rename(columns={"text":"tweet"})
preProcess(tweets_df)

In [33]:
tweets= tweets_df['tweet'].copy()

# Predicting and Analysis

In [34]:
import pickle

with open('Models/nb_lexicon_model.sav', mode='rb') as f:
    nb_lex_count_model = pickle.load(f)

In [35]:
df2 = pd.read_csv('Datasets/tweets_train.csv')
df1 = pd.read_csv('Datasets/NU_EG_Twitter_corpus_train.csv')
df2.rename(columns={"text":"tweet"},inplace=True)
df = df2.append(df1,ignore_index = True)
preProcess(df)
tweets2 = df['tweet'].copy()

C:\Users\youss\AppData\Local\Temp\ipykernel_12624\1748729805.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df2.append(df1,ignore_index = True)


In [36]:
tokens1 = set(nltk.tokenize.wordpunct_tokenize(' '.join(tweets.to_numpy().flatten())))
tokens2 = set(nltk.tokenize.wordpunct_tokenize(' '.join(tweets2.to_numpy().flatten())))

In [37]:
print(len(set(tokens1)), len(set(tokens2)))

7834 28109


In [45]:
print(set(tokens1).difference(set(tokens2)))

{'تتحل', 'بيطلعو', 'سميث', 'بحواجب', 'اتصاحب', 'مرادي', '201280959900', 'کبار', 'تنبهك', 'الفني', 'مخطوب', 'ميجابايتس', 'VaX8eFJwFR', 'yjLswau0n5', 'حبق', 'يوصلي', '0iV5isxaYU', 'vkHulSJxNU', 'مولتن', 'فتحتش', 'مفهمتش', 'صحراوي', 'DqusgwaOOy', 'antique', 'التكنولوجيا', 'ركبو', 'تيم', 'هنوصل', 'nasb', 'اقبض', 'مايدور', 'مباركا', 'كتكوت', 'الفرنسيه223', 'يوفنتوس', 'هاج', 'صططلاق', 'لازياء', 'U1WqFSjLWe', 'حقنا', 'بتستظرف', 'MmfvP3O2yC', 'Mobinil', 'fixed', 'العميل', 'بيتهيالي', 'كدابه', 'EPfMiUb9Qz', 'فشخان', 'ZxubIVlnUb', 'هيقول', 'مشربش', 'English', 'jGRvPP8Igp', 'يني', 'معصب', 'Stay', 'القريه', 'ميشوفهوش', 'انفصلت', 'lRGd1StZiq', 'شريكات', 'مجدد', 'معيد', 'شريهان', 'wifi', 'مجددوش', 'هيجيل', 'اتلقطت', 'اورانچ', '24RANLK4sF', 'يبعتول', 'مستعجل', 'شركت', 'يسمعو', 'تاخيرات', 'اتسرقت', 'FORZA', '٣٦٥', 'تخصمو', 'ڤودافون', 'خالك', 'اتمسك', 'TECNO', 'بتفسح', 'fayoum', 'اعرفش', 'جاسمي', 'مبيوصليش', 'BpILObYEJZ', 'خطوط', 'leader', 'mshmsh', 'متضرر', 'كوالتي', 'مقالهاش', 'NcwhCGFp7H', 'خرر', 'ع

In [39]:
def tweet_filter(tweet: str, bucket: set[str]) -> bool:
    tokens = set(nltk.tokenize.wordpunct_tokenize(tweet))
    inclusion_ratio =  len(tokens.intersection(bucket)) / len(tokens)
    return inclusion_ratio >= 0.6


filtered_tweets = tweets.to_numpy()[[tweet_filter(x, tokens2) for x in tweets.to_numpy()]]

In [40]:
print(len(filtered_tweets))

4504
